In [1]:
import pandas as pd
import numpy as np; np.random.seed(13)

train_df = pd.read_csv('../input/train.csv')
X, y = train_df.iloc[:, 0:len(train_df.columns) - 1], train_df.iloc[:, -1]

test_df = pd.read_csv('../input/test.csv')
X_test, y_test = test_df.iloc[:, 0:len(test_df.columns) -1], test_df.iloc[:, -1]

In [4]:
from sklearn.model_selection import GridSearchCV

from sklearn.neural_network import MLPClassifier

parameters = {
    'hidden_layer_sizes': [(50,), (100,), (200,)],
}

selector = GridSearchCV(MLPClassifier(), parameters, scoring='accuracy') # we only care about accuracy here
selector.fit(X, y)

print('Best parameter set found:')
print(selector.best_params_)
print('Detailed grid scores:')
means = selector.cv_results_['mean_test_score']
stds = selector.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, selector.cv_results_['params']):
    print('%0.3f (+/-%0.03f) for %r' % (mean, std * 2, params))
    print()

Best parameter set found:
{'hidden_layer_sizes': (50,)}
Detailed grid scores:
0.913 (+/-0.008) for {'hidden_layer_sizes': (50,)}

0.907 (+/-0.034) for {'hidden_layer_sizes': (100,)}

0.909 (+/-0.016) for {'hidden_layer_sizes': (200,)}



In [45]:
from sklearn.svm import SVC

svc = SVC(kernel='linear', C=1, probability=True).fit(X, y)
mlp = MLPClassifier(hidden_layer_sizes=(50,)).fit(X, y)

In [47]:
class_dict = {i: svc.classes_[i] for i in range(6)}
class_dict

{0: 'LAYING',
 1: 'SITTING',
 2: 'STANDING',
 3: 'WALKING',
 4: 'WALKING_DOWNSTAIRS',
 5: 'WALKING_UPSTAIRS'}

In [48]:
def get_int_pred(prob_prediction):
    return sum(prob_prediction[i] * i for i in range(6))

svc_int_pred = np.apply_along_axis(get_int_pred, 1, svc.predict_proba(X_test))
mlp_int_pred = np.apply_along_axis(get_int_pred, 1, mlp.predict_proba(X_test))

In [49]:
svc_int_pred

array([ 2.00002583,  1.97060413,  1.88881092, ...,  4.99688445,
        4.99259536,  4.91267618])

In [50]:
mlp_int_pred

array([ 1.99981   ,  1.95034156,  1.99220073, ...,  4.99648951,
        4.98526921,  4.82132932])

In [82]:
weight = 1

ensemble_int_pred = weight * svc_int_pred + (1 - weight) * mlp_int_pred
ensemble_int_pred

array([ 2.00002583,  1.97060413,  1.88881092, ...,  4.99688445,
        4.99259536,  4.91267618])

In [83]:
np.round_(ensemble_int_pred)

array([ 2.,  2.,  2., ...,  5.,  5.,  5.])

In [84]:
ensemble_pred = np.vectorize(class_dict.get)(np.round_(ensemble_int_pred))
ensemble_pred

array(['STANDING', 'STANDING', 'STANDING', ..., 'WALKING_UPSTAIRS',
       'WALKING_UPSTAIRS', 'WALKING_UPSTAIRS'],
      dtype='<U18')

In [85]:
from sklearn.metrics import accuracy_score

print('Accuracy:', accuracy_score(ensemble_pred, y_test))

Accuracy: 0.957583983712
